In [1]:
import pandas as pd

# Ruta completa y correcta
archivo = r"C:\Users\guerr\Documents\AnalisisDeDatos\Dahell\data\historial de cartera.xlsx"

# Intentar cargar
try:
    df = pd.read_excel(archivo, engine='openpyxl')
    print("✅ Archivo cargado con éxito")
except FileNotFoundError:
    print("❌ No se encontró el archivo, verifica la ruta.")
except Exception as e:
    print("❌ Otro error:", e)

df.head()  # Mostrar las primeras filas del DataFrame
df.info()

✅ Archivo cargado con éxito
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 292 non-null    int64  
 1   FECHA                              292 non-null    object 
 2   TIPO                               292 non-null    object 
 3   MONTO                              292 non-null    float64
 4   MONTO PREVIO                       292 non-null    float64
 5   ORDEN ID                           274 non-null    float64
 6   NUMERO DE GUIA                     274 non-null    object 
 7   DESCRIPCIÓN                        292 non-null    object 
 8   USUARIO QUE REALIZA EL MOVIMIENTO  21 non-null     object 
 9   CUENTA                             274 non-null    object 
 10  CONCEPTO DE RETIRO                 1 non-null      object 
dtypes: float64(3), int64(1), objec

In [2]:
df.drop(columns=["CONCEPTO DE RETIRO", "USUARIO QUE REALIZA EL MOVIMIENTO","CUENTA", "NUMERO DE GUIA"], inplace=True)

In [4]:
import pandas as pd
import numpy as np

# Clasificar los movimientos por descripción
def clasificar_movimiento(desc):
    if pd.isna(desc):
        return 'OTRO'
    desc = desc.upper()
    if 'GANANCIA' in desc:
        return 'GANANCIA'
    elif 'FLETE' in desc:
        return 'FLETE'
    elif 'PUBLICIDAD' in desc:
        return 'PUBLICIDAD'
    elif 'RETIRO' in desc:
        return 'RETIRO'
    elif 'SALIDA' in desc:
        return 'SALIDA'
    elif 'ENTRADA' in desc:
        return 'ENTRADA'
    else:
        return 'OTRO'

df['CATEGORIA_MOVIMIENTO'] = df['DESCRIPCIÓN'].apply(clasificar_movimiento)

# Bandera si es ingreso o egreso
df['ES_INGRESO'] = df['TIPO'].str.upper().str.contains('ENTRADA')
df['ES_EGRESO'] = df['TIPO'].str.upper().str.contains('SALIDA')

# Calcular el saldo posterior (simplemente monto previo +/- monto)
df['SALDO_POSTERIOR'] = np.where(
    df['ES_INGRESO'],
    df['MONTO PREVIO'] + df['MONTO'],
    df['MONTO PREVIO'] - df['MONTO']
)

# Limpiar y estandarizar número de orden
df['ORDEN_ID_LIMPIO'] = df['ORDEN ID'].fillna(0).astype(int)


In [7]:
# Verificar nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
ID                       0
FECHA                    0
TIPO                     0
MONTO                    0
MONTO PREVIO             0
ORDEN ID                18
DESCRIPCIÓN              0
CATEGORIA_MOVIMIENTO     0
ES_INGRESO               0
ES_EGRESO                0
SALDO_POSTERIOR          0
ORDEN_ID_LIMPIO          0
dtype: int64


In [11]:
# Nueva columna que indica si el movimiento está relacionado con una orden o no
df['TIENE_ORDEN'] = df['ORDEN ID'].notna()

# Nueva columna que describe mejor el tipo de movimiento según si tiene orden o no
df['CATEGORIA_MOVIMIENTO'] = df.apply(
    lambda row: 'ASOCIADO A ORDEN' if row['TIENE_ORDEN']
    else ('RETIRO GENERAL' if row['TIPO'] == 'SALIDA' else 'INGRESO GENERAL'),
    axis=1
)


In [14]:
# Convertir la columna FECHA a tipo datetime
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d-%m-%Y %H:%M')

# Extraer nuevas variables temporales
df['DIA'] = df['FECHA'].dt.date
df['HORA'] = df['FECHA'].dt.time
df['DIA_SEMANA'] = df['FECHA'].dt.day_name()
df['MES'] = df['FECHA'].dt.month
df['AÑO'] = df['FECHA'].dt.year


In [15]:
import re

def extraer_motivo(texto):
    if "GANANCIA" in texto:
        return "GANANCIA"
    elif "FLETE" in texto:
        return "FLETE"
    elif "NUEVA ORDEN" in texto:
        return "NUEVA ORDEN"
    elif "RETORNO" in texto:
        return "RETORNO"
    elif "AJUSTE" in texto:
        return "AJUSTE"
    elif "BONIFICACIÓN" in texto:
        return "BONIFICACIÓN"
    else:
        return "OTRO"

df['MOTIVO'] = df['DESCRIPCIÓN'].apply(extraer_motivo)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    292 non-null    int64         
 1   FECHA                 292 non-null    datetime64[ns]
 2   TIPO                  292 non-null    object        
 3   MONTO                 292 non-null    float64       
 4   MONTO PREVIO          292 non-null    float64       
 5   ORDEN ID              274 non-null    float64       
 6   DESCRIPCIÓN           292 non-null    object        
 7   CATEGORIA_MOVIMIENTO  292 non-null    object        
 8   ES_INGRESO            292 non-null    bool          
 9   ES_EGRESO             292 non-null    bool          
 10  SALDO_POSTERIOR       292 non-null    float64       
 11  ORDEN_ID_LIMPIO       292 non-null    int64         
 12  TIENE_ORDEN           292 non-null    bool          
 13  DIA                 

In [17]:
df.to_csv('../output/historial_cartera.csv', index=False, encoding='utf-8-sig')